In [1]:
import os
import time
from dotenv import load_dotenv

# Librerías de LangChain y Chroma
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# Cargamos variables de entorno (.env)
load_dotenv()

# --- CONFIGURACIÓN INICIAL ---
CARPETA_DB = "chroma_db"
MODELO_EMBEDDINGS = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

print("⚙️  Configurando Agente Inteligente USACH...")

c:\AgentesIA\MauricIA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⚙️  Configurando Agente Inteligente USACH...


In [ ]:
# 1. Configuración del LLM (Ollama)
try:
    llm = ChatOllama(
        # Valor por defecto seguro
        base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
        # Asegúrate de tener el modelo correcto
        model=os.getenv("OLLAMA_MODEL", "llama3"),
        temperature=0.1,  # Temperatura baja para respuestas factuales
        num_predict=300,  # Un poco más de espacio para responder
    )
    print(f"✓ Cerebro cargado: {llm.model}")

except Exception as e:
    print(f"✗ Error conectando a Ollama: {e}")
    exit()

# 2. Conexión a la Base de Datos Vectorial (Tu "Memoria")
# IMPORTANTE: Usamos el MISMO modelo de embeddings que usamos para crear la base de datos
print("🔌 Conectando a la base de conocimiento local...")

embedding_function = HuggingFaceEmbeddings(model_name=MODELO_EMBEDDINGS)

if os.path.exists(CARPETA_DB):
    vector_db = Chroma(
        persist_directory=CARPETA_DB,
        embedding_function=embedding_function
    )
    print("✓ Base de datos ChromaDB conectada.")
else:
    print("✗ ERROR CRÍTICO: No encuentro la carpeta 'chroma_db'. Ejecuta primero 'crear_cerebro.py'")
    exit()

# Configuración del Retriever (Buscador)
# k=3 significa que traerá los 3 fragmentos más relevantes
retriever = vector_db.as_retriever(search_kwargs={"k": 3})


# --- WARM-UP (CALENTAMIENTO DE MOTORES) ---
print("\n🔥 Iniciando secuencia de calentamiento (para evitar esperas)...")

# 1. Calentar el LLM (Ollama)
try:
    # Le pedimos algo muy corto para que cargue en memoria sin gastar tiempo generando mucho texto
    print("   - Cargando modelo LLM en VRAM...", end="", flush=True)
    llm.invoke("test")
    print(" [LISTO]")
except Exception as e:
    print(f" [ERROR LLM: {e}]")

# 2. Calentar el Buscador (Embeddings)
# Esto carga el modelo sentence-transformers en memoria
try:
    print("   - Cargando sistema de búsqueda semántica...", end="", flush=True)
    retriever.invoke("test")
    print(" [LISTO]")
except Exception as e:
    print(f" [ERROR RETRIEVER: {e}]")

print("✓ Sistema 100% operativo y listo para recibir usuarios.\n")


# --- NUEVA FUNCIÓN PURA (Lógica Testable) ---
def obtener_respuesta_agente(user_input):
    """
    Recibe el texto del usuario y devuelve la respuesta del agente como string.
    No imprime nada, solo procesa.
    """
    
    # 1. DETECTOR DE SALUDOS
    palabras_clave = [
    "hola", "holi", "wena", "wenas", "buenas", "buenos",
    "buen día", "buen dia", "buenas tardes", "buenas noches",
    "saludos", "un saludo", "saludo",
    "qué tal", "que tal", "como estas", "cómo estás", "cómo estai", "como estai",
    "hey", "hi", "hello", "yo", "sup"
]

    es_saludo = any(p in user_input.lower() for p in palabras_clave) and len(user_input.split()) < 6

    # Definimos los prompts (Los mismos que ya tienes corregidos)
    system_prompt_base = (
        "Eres un experto en admisiones de Postgrado de la USACH llamado asi tal cual MauricIA.\n"
        "Atiendes a TODOS los estudiantes. NO te limites a un grupo.\n"
        "Tu misión es extraer información EXACTA del contexto proporcionado. NO inventes ni asumas.\n"
        "DEFINICIONES CRÍTICAS:\n"
        "- MATRÍCULA: Costo semestral administrativo, valor en el contexto.\n"
        "- ARANCEL: Costo anual de estudios, valor en el contexto.\n"
        "REGLAS:\n"
        "1. Cita TEXTUALMENTE los montos.\n"
        "2. Si la info no está, di: 'No encuentro ese dato específico'.\n"
    )
    
    system_prompt_saludo = (
        "Eres MauricIA, el chatbot oficial de porgramas de Postgrados USACH. "
        "Ayudas a estudiantes chilenos y extranjeros por igual. "
        "Tu trabajo es saludar amablemente y pedir una consulta académica. "
    )

    full_response = ""
    messages = []

    try:
        if es_saludo:
            # CAMINO A: Saludo
            messages = [
                SystemMessage(content=system_prompt_saludo),
                HumanMessage(content=user_input)
            ]
        else:
            # CAMINO B: RAG
            docs_relacionados = retriever.invoke(user_input)
            context_text = "\n\n".join([d.page_content for d in docs_relacionados])
            
            messages = [
                SystemMessage(content=system_prompt_base),
                HumanMessage(content=f"CONTEXTO:\n{context_text}\n\nPREGUNTA:\n{user_input}")
            ]

        # Invocamos al LLM (sin stream para el test, o acumulando el stream)
        # Para tests es mejor invoke directo, pero para mantener tu logica usamos invoke
        response = llm.invoke(messages)
        return response.content

    except Exception as e:
        return f"Error interno: {str(e)}"

# --- LA INTERFAZ DE USUARIO (El Chat) ---
def chatbot_streaming():
    print("\n🎓 === ASISTENTE DE POSTGRADOS USACH ===")
    print("Escribe 'salir' para cerrar.\n")

    while True:
        user_input = input("\n🧑 Tú: ").strip()
        if user_input.lower() in ["salir", "exit"]: break
        if not user_input: continue

        print("\n🤖 Asistente: ", end="", flush=True)
        
        # Llamamos a la lógica
        respuesta = obtener_respuesta_agente(user_input)
        
        # Simulamos streaming para el usuario (opcional)
        print(respuesta) 

if __name__ == "__main__":
    chatbot_streaming()